**Author:** Pedro Evaristo de Oliveira - 20/10/2025

GoodThought NGO has been a catalyst for positive change, focusing its efforts on education, healthcare, and sustainable development to make a significant difference in communities worldwide. With this mission, GoodThought has orchestrated an array of assignments aimed at uplifting underprivileged populations and fostering long-term growth.

This project offers a hands-on opportunity to explore how data-driven insights can direct and enhance these humanitarian efforts. In this project, you'll engage with the GoodThought PostgreSQL database, which encapsulates detailed records of assignments, funding, impacts, and donor activities from 2010 to 2023. This comprehensive dataset includes:

- **`Assignments`:** Details about each project, including its name, duration (start and end dates), budget, geographical region, and the impact score.
- **`Donations`:** Records of financial contributions, linked to specific donors and assignments, highlighting how financial support is allocated and utilized.
- **`Donors`:** Information on individuals and organizations that fund GoodThought’s projects, including donor types.


**Query 1:** List the top five assignments based on total value of donations, categorized by donor type. The output should include four columns: 1) assignment_name, 2) region, 3) rounded_total_donation_amount rounded to two decimal places, and 4) donor_type, sorted by rounded_total_donation_amount in descending order. 


In [2]:
-- This query retrieves the top 5 assignment and donor type combinations with the highest total donation amounts.
-- It joins the assignments, donations, and donors tables to aggregate donation data.
-- The results are grouped by assignment name, donor type, and region, and ordered by the total donation amount in descending order.

SELECT 
    a.assignment_name AS assignment_name,                -- Name of the assignment
    donors.donor_type AS donor_type,                     -- Type of the donor (e.g., individual, corporate)
    a.region AS region,                                  -- Region associated with the assignment
    ROUND(SUM(donat.amount),2) AS rounded_total_donation_amount  -- Total donation amount (rounded to 2 decimals)
FROM 
    public.assignments AS a
INNER JOIN 
    public.donations AS donat
    ON a.assignment_id = donat.assignment_id             -- Join donations to assignments by assignment_id
INNER JOIN 
    public.donors AS donors
    ON donat.donor_id = donors.donor_id                  -- Join donors to donations by donor_id
GROUP BY 
    a.assignment_name, donors.donor_type, a.region       -- Group by assignment, donor type, and region
ORDER BY 
    rounded_total_donation_amount DESC                   -- Order by total donation amount, highest first
LIMIT 5;                                                 -- Limit results to top 5 rows

,assignment_name,donor_type,region,rounded_total_donation_amount
0,Assignment_3033,Individual,East,3840.66
1,Assignment_300,Organization,West,3133.98
2,Assignment_4114,Organization,North,2778.57
3,Assignment_1765,Organization,West,2626.98
4,Assignment_268,Individual,East,2488.69





**Query 2:** Identify the assignment with the highest impact score in each region, ensuring that each listed assignment has received at least one donation. The output should include four columns: 1) assignment_name, 2) region, 3) impact_score, and 4) num_total_donations, sorted by region in ascending order. Include only the highest-scoring assignment per region, avoiding duplicates within the same region. 

In [3]:
-- This query finds the top-impact assignment in each region, along with its total donation amount.

-- A CTE (Common Table Expression) to rank assignments by impact_score within each region.
WITH ranked_assignments AS (
    SELECT
        a.region,                        -- The region of the assignment
        a.assignment_id,                 -- Unique identifier for the assignment
        a.assignment_name,               -- Name of the assignment
        a.impact_score,                  -- Impact score of the assignment
        SUM(d.amount) AS num_total_donations, -- Total donation amount for this assignment
        ROW_NUMBER() OVER (
            PARTITION BY a.region        -- Restart row numbering for each region
            ORDER BY a.impact_score DESC, -- Highest impact_score first
                     a.assignment_id ASC  -- Tie-breaker: lowest assignment_id first
        ) AS rn                          -- Row number within each region
    FROM public.assignments AS a
    INNER JOIN public.donations AS d
        ON a.assignment_id = d.assignment_id -- Join assignments with their donations
    GROUP BY a.region, a.assignment_id, a.assignment_name, a.impact_score
        -- Group by all non-aggregated columns
)

-- Select the top-ranked (highest impact_score) assignment per region.
SELECT
    region,             -- Region name
    assignment_name,    -- Assignment name
    impact_score,       -- Impact score
    num_total_donations -- Total donation amount for this assignment
FROM ranked_assignments
WHERE rn = 1;           -- Only include the top assignment per region

,region,assignment_name,impact_score,num_total_donations
0,East,Assignment_316,10.00,530.81
1,North,Assignment_2253,9.99,872.03
2,South,Assignment_3547,10.00,880.15
3,West,Assignment_2794,9.99,691.58


**Source content:** Datacamp.com